In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

print(torch.__version__)

1.12.1


In [2]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

In [8]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'instruction'  # instruction
        pattern_output = 'output'  # response
            
        list_data_dict = []
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file2:
            for line in json_file2:
                list_data_dict.append(json.loads(line))

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [4]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [9]:
train_dataset = SFT_dataset(
    data_path_1_SFT='./data/KoAlpaca_v1.1.jsonl', 
    tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  8000, 13532,  9863, 10856, 23169, 14668,  8084,
          406,  9394, 45941, 17415, 13961, 14668, 13675,   375,   378,   378,
          378, 41951,   454,  9549, 20549,   383,  8142,  7192, 14914,  8000,
        13532, 15694, 11039, 17752, 15380,  9679, 21154, 45941, 17415, 17752,
        13961,  9679, 21154,  9051,   375, 30895, 40921, 10066,  8022,  9552,
        48397,  8711,  9033,  9129, 20661, 11296,  9018, 45211,  9167, 35482,
        25579,  9846, 32240,  9164, 13532, 15694, 11039, 15380,  9679, 21154,
        45941, 17415,  9306,  9633, 10292, 12817,  7643,  9023, 47518, 10748,
         9779, 13961,  9679, 21154, 15957,  9164, 13532, 17752, 15380, 13748,
        11794, 45941, 17415, 17752, 13961,  9679, 10542,   375, 16773,  9489,
        12817,  7643,   401, 29755,  9566,  9313, 19970,  7965,  7513,  8137,
         9025,  9019,  7055,  8084,   406,  9051,   375,

In [10]:
training_args = TrainingArguments(
    output_dir="./logs",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=4,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True,
    optim="adafactor",
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [11]:
trainer.train()
model.save_pretrained('./models')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.624400


In [12]:
generator = pipeline('text-generation', model='./models', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):불고기용 고기는 쇠고기, 돼지고기, 닭고기, 오리고기 등 다양한 부위를 사용합니다. 불고기용 고기의 종류에 따라 소고기, 돼지고기, 양고기, 닭고기 등 다양한 부위를 사용할 수 있습니다. 



### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):1956년부터 1981년까지 미국 대통령 선거에서는 프랭클린 루스벨트가 44대 부통령직을 수행했습니다. 그러나 이 기간 동안 존 F. 케네디가 대통령에 취임하지 않았습니다. 



### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):시카고 오 헤어 국제공항은 미국 플로리다주 케이프 커내버럴에 위치해 있습니다. 이 공항은 미국의 대표적인 관광지 중 하나입니다. 이 공항은 뉴욕, 뉴저지, 애틀랜타 등 여러 도시에 위치하고 있습니다. 이 공항의 이름은 "Chicago Airport Hotel"입니다. 이 공항

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):미세먼지 어는 이유는 중국발 스모그 때문입니다. 중국에서 날아온 스모그가 우리나라로 유입되면서 공기 중에 먼지가 쌓이기 시작했습니다. 이로 인해 대기 중 먼지가 우리 눈에 들어오게 됩니다. 이에 따라 미세먼지 농도가 높아지게 됩니다. 


